In [2]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor #Es un regresor utiliza árboles de decisión para poder predecir el valor de una variable continua.
from sklearn.model_selection import (cross_val_score, KFold)

In [3]:
dataset = pd.read_csv('Datasets/felicidad.csv')
dataset.head()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [4]:
X = dataset.drop(['country', 'score'], axis=1)
Y = dataset['score']

## Implementación básica y  rápida de Cross Validation

In [5]:
#Instanciamos nuestro modelo

model=DecisionTreeRegressor()
#Ahora, generalmente aquí empleamos ".fit()", pero esto ya lo incluye los seleccionadores que utilizaremos ahora
score = cross_val_score(model, X, Y, scoring= 'neg_mean_squared_error') #Función recomendada cuando queremos realizar una prueba rápida.
#Esta función tiene como parámetros a nuestro modelo, a nuestros features"X" y a nuestro target "Y".
#Además de ello, en el parámetro "scoring" le estamos indicando qué metodo emplearemos para el scoring de nuestro cross validation.

print(score)

[-0.41966018 -0.06450492 -0.08829864 -0.07015596 -0.41181261]


In [6]:
score = cross_val_score(model, X, Y,cv=3, scoring= 'neg_mean_squared_error')
print(np.abs(np.mean(score)))

0.5689682151858332


Obtenemos un arreglo de score **negativos**, pero tranquilo esto es normal es la forma estandar cuando trabajamos con **cross validation**

#### NOTA:
El score es un **arreglo de errores negativos medios cuadrados** (es decir, cuanto mas pequeño en valor absoluto, mejor se ajusta el modelo a los datos) como salida del cross_val_score, este resultado se da ya que el modelo fue separado *"cv"* veces (en este caso 5 al principio y luego 3 -cuando ingresamos cv=3 -) en set de datos de entrenamiento y prueba, en lo que se puede notar que particiones fueron mas satisfactorias.
Ahora al aplicar el promedio y el valor absoluto *(con np.mean y np.abs)* , puedes observar el error medio cuadrado promedio calculado a partir de las salidas score que evalúan la adaptación promedio del modelo a los datos.

##### NOTA2:
También *"cross_val_score"* puede darte **como salida otro tipo de error**, a través del parámetro scoring.
<cross_val_score(model,X,y, cv=5,scoring='r2')>

## Cross Validation análisis más detallado 

Para ello Utilizaremos K-Folds

In [39]:
#Instanciamos nuestro seleccionador

kf = KFold(n_splits=3, shuffle=True, random_state=42)
# con "n_splits" indicamos la cantidad de pliegues que hara este seleccionador a nuestros datos. Recuerda que lo divide en "cv" pliegues y escoge uno para testeo y los restantes para entrenamiento
# con "shuffle" activamos para nuestros datos se organicen aleatoriamente "True" o en orden "False"
# El "random_state" ya vimos anteriormente para qué lo usamos

In [40]:
#Ahora recorreremos el split en un for para que nos devuelva los índices.
#Recuerda que el método "split" de la clase KFold nos devuelve los índices de entrenamiento y de prueba.
for train, test in kf.split(dataset): #dara "n_splits" pasadas a nuestro dataset y los separatá en los pliegues de datos y prueba
    print(train) 
    print(test)

[  0   1   2   3   4   5   6   7   8  10  13  14  16  17  20  21  23  25
  28  32  33  34  35  37  38  39  40  41  43  44  46  47  48  49  50  52
  53  54  57  58  59  61  62  63  64  67  70  71  72  73  74  77  80  83
  87  88  89  91  92  94  97  98  99 100 101 102 103 104 105 106 107 108
 110 111 112 113 114 115 116 120 121 123 125 127 128 129 130 132 134 135
 136 139 140 143 144 145 146 148 149 150 151 152 154]
[  9  11  12  15  18  19  22  24  26  27  29  30  31  36  42  45  51  55
  56  60  65  66  68  69  75  76  78  79  81  82  84  85  86  90  93  95
  96 109 117 118 119 122 124 126 131 133 137 138 141 142 147 153]
[  1   2   3   6   8   9  11  12  13  14  15  17  18  19  20  21  22  24
  26  27  29  30  31  36  37  38  42  45  48  50  51  52  54  55  56  57
  58  59  60  63  65  66  68  69  71  72  74  75  76  78  79  81  82  83
  84  85  86  87  88  89  90  91  92  93  95  96  99 100 102 103 106 107
 109 112 115 116 117 118 119 120 121 122 124 126 128 129 130 131 132 133
 135

Luego  de obtener los índices, estos los podemos asignar a un modelo como X_train, X_test, Y_train e Y_test

 X_train, X_test = X**[train]**, X**[test]**
 
 y_train, y_test = Y**[train]**, Y**[test]**
 
 Una vez hecho ello, nosotros pasaremos a ingresar esos datos a diversos modelos o a un solo modelo para poder obtener nuestros scores.

### OBS:
Lo que hicimos en lo anterior es revisar más a profunco lo que hace Cross Validation.
